## Import Libraries

In [ ]:
!pip install transformers -q
import pandas as pd
import numpy as np
import tensorflow as tf
import torch
from torch.nn import BCEWithLogitsLoss, BCELoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score
import pickle
from tqdm import tqdm, trange
from ast import literal_eval
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla K80'

## Load and Preprocess Training Data

In [6]:
import pandas as pd
from itertools import chain

df_initial = pd.read_excel('/content/train.xlsx')
df = pd.read_excel('/content/train.xlsx')

df = df.drop(['IOB Slot tags'], axis = 1)

df.head()

,utterances,Core Relations
0,who plays luke on star wars new hope,movie.starring.actor movie.starring.character
1,show credits for the godfather,movie.starring.actor
2,who was the main actor in the exorcist,movie.starring.actor
3,who played dory on finding nemo,movie.starring.actor movie.starring.character
4,who was the female lead in resident evil,movie.starring.actor actor.gender


In [7]:
relations_vocab = [str(i).split() for i in df['Core Relations']]
relations_vocab = list(chain.from_iterable(relations_vocab))
relations_vocab = list(set(relations_vocab))

relations2index = {j:i for i, j  in enumerate(relations_vocab)}
index2relations = {i:j for i, j  in enumerate(relations_vocab)}

print(relations2index)
print(index2relations)

{'movie.production_companies': 0, 'movie.gross_revenue': 1, 'movie.initial_release_date': 2, 'movie.country': 3, 'gr.amount': 4, 'person.date_of_birth': 5, 'movie.locations': 6, 'movie.subjects': 7, 'actor.gender': 8, 'movie.directed_by': 9, 'movie.starring.character': 10, 'movie.music': 11, 'movie.genre': 12, 'movie.language': 13, 'movie.rating': 14, 'movie.starring.actor': 15, 'movie.produced_by': 16, 'nan': 17, 'movie.estimated_budget': 18}
{0: 'movie.production_companies', 1: 'movie.gross_revenue', 2: 'movie.initial_release_date', 3: 'movie.country', 4: 'gr.amount', 5: 'person.date_of_birth', 6: 'movie.locations', 7: 'movie.subjects', 8: 'actor.gender', 9: 'movie.directed_by', 10: 'movie.starring.character', 11: 'movie.music', 12: 'movie.genre', 13: 'movie.language', 14: 'movie.rating', 15: 'movie.starring.actor', 16: 'movie.produced_by', 17: 'nan', 18: 'movie.estimated_budget'}


In [8]:
df.iloc[1]['Core Relations']

'movie.starring.actor'

In [9]:
for i in relations2index.keys():

  df[f'{i}'] = [0]*len(df)

df.head()

,utterances,Core Relations,movie.production_companies,movie.gross_revenue,movie.initial_release_date,movie.country,gr.amount,person.date_of_birth,movie.locations,movie.subjects,...,movie.directed_by,movie.starring.character,movie.music,movie.genre,movie.language,movie.rating,movie.starring.actor,movie.produced_by,nan,movie.estimated_budget
0,who plays luke on star wars new hope,movie.starring.actor movie.starring.character,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,show credits for the godfather,movie.starring.actor,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,who was the main actor in the exorcist,movie.starring.actor,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,who played dory on finding nemo,movie.starring.actor movie.starring.character,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,who was the female lead in resident evil,movie.starring.actor actor.gender,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
df.iloc[1]['movie.starring.actor']

0

In [11]:
for j in range(len(df)):

  relation = str(df.iloc[j]['Core Relations']).split()
  
  for k in relation:

    df.at[j, f'{k}'] = 1

df = df.drop(['Core Relations', 'nan'], axis = 1)

df.head()

,utterances,movie.production_companies,movie.gross_revenue,movie.initial_release_date,movie.country,gr.amount,person.date_of_birth,movie.locations,movie.subjects,actor.gender,movie.directed_by,movie.starring.character,movie.music,movie.genre,movie.language,movie.rating,movie.starring.actor,movie.produced_by,movie.estimated_budget
0,who plays luke on star wars new hope,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
1,show credits for the godfather,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,who was the main actor in the exorcist,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,who played dory on finding nemo,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
4,who was the female lead in resident evil,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0


In [12]:
 df.isnull().values.any()

False

Dataset will be tokenized then split into training and validation sets. The validation set will be used to monitor training. For testing a separate test set will be loaded for analysis.

In [13]:
cols = df.columns
label_cols = list(cols[1:])
num_labels = len(label_cols)
print('Label columns: ', label_cols)

Label columns:  ['movie.production_companies', 'movie.gross_revenue', 'movie.initial_release_date', 'movie.country', 'gr.amount', 'person.date_of_birth', 'movie.locations', 'movie.subjects', 'actor.gender', 'movie.directed_by', 'movie.starring.character', 'movie.music', 'movie.genre', 'movie.language', 'movie.rating', 'movie.starring.actor', 'movie.produced_by', 'movie.estimated_budget']


In [14]:
print('Count of 1 per label: \n', df[label_cols].sum(), '\n') # Label counts, may need to downsample or upsample
print('Count of 0 per label: \n', df[label_cols].eq(0).sum())

Count of 1 per label: 
 movie.production_companies     93
movie.gross_revenue            31
movie.initial_release_date    180
movie.country                 222
gr.amount                       5
person.date_of_birth            4
movie.locations                 3
movie.subjects                 94
actor.gender                    8
movie.directed_by             335
movie.starring.character       22
movie.music                     3
movie.genre                   128
movie.language                218
movie.rating                  208
movie.starring.actor          340
movie.produced_by             173
movie.estimated_budget         68
dtype: int64 

Count of 0 per label: 
 movie.production_companies    2160
movie.gross_revenue           2222
movie.initial_release_date    2073
movie.country                 2031
gr.amount                     2248
person.date_of_birth          2249
movie.locations               2250
movie.subjects                2159
actor.gender                  2245
movie.dire

In [15]:
len(df)

2253

In [16]:
df = df.sample(frac=1).reset_index(drop=True) #shuffle rows

In [17]:
len(df)

2253

In [18]:
df['one_hot_labels'] = list(df[label_cols].values)
df.head()

,utterances,movie.production_companies,movie.gross_revenue,movie.initial_release_date,movie.country,gr.amount,person.date_of_birth,movie.locations,movie.subjects,actor.gender,movie.directed_by,movie.starring.character,movie.music,movie.genre,movie.language,movie.rating,movie.starring.actor,movie.produced_by,movie.estimated_budget,one_hot_labels
0,who were the crew for the campaign,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,who directed soul surfer,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
2,what actors appeared in the movie hitch,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,when is iron man 3 coming out,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,i want to watch a drama made in china,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [19]:
labels = list(df.one_hot_labels.values)
comments = list(df.utterances.values)

Load the pretrained tokenizer that corresponds to your choice in model. e.g.,

```
BERT:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True) 

XLNet:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=False) 

RoBERTa:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=False)
```


In order to avoid memory issues with Google Colab, I enforce a max_length of 100 tokens. Note that some sentences may not adequately represent each label because of this.

In [20]:
max_length = 100
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True) # tokenizer
encodings = tokenizer.batch_encode_plus(comments,max_length=max_length,pad_to_max_length=True) # tokenizer's encoding method
print('tokenizer outputs: ', encodings.keys())

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


tokenizer outputs:  dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [21]:
input_ids = encodings['input_ids'] # tokenized and encoded sentences
token_type_ids = encodings['token_type_ids'] # token type ids
attention_masks = encodings['attention_mask'] # attention masks

In [22]:
# Identifying indices of 'one_hot_labels' entries that only occur once - this will allow us to stratify split our training data later
label_counts = df.one_hot_labels.astype(str).value_counts()
one_freq = label_counts[label_counts==1].keys()
one_freq_idxs = sorted(list(df[df.one_hot_labels.astype(str).isin(one_freq)].index), reverse=True)
print('df label indices with only one instance: ', one_freq_idxs)

df label indices with only one instance:  [1969, 1941, 1544, 1409, 1370, 1351, 1098, 952, 805, 514, 483, 473, 367, 58, 40]


In [23]:
# Gathering single instance inputs to force into the training set after stratified split
one_freq_input_ids = [input_ids.pop(i) for i in one_freq_idxs]
one_freq_token_types = [token_type_ids.pop(i) for i in one_freq_idxs]
one_freq_attention_masks = [attention_masks.pop(i) for i in one_freq_idxs]
one_freq_labels = [labels.pop(i) for i in one_freq_idxs]

Be sure to handle all classes during validation using "stratify" during train/validation split:

In [24]:
# Use train_test_split to split our data into train and validation sets

train_inputs, validation_inputs, train_labels, validation_labels, train_token_types, validation_token_types, train_masks, validation_masks = train_test_split(input_ids, labels, token_type_ids,attention_masks,
                                                            random_state=2020, test_size=0.10, stratify = labels)

# Add one frequency data to train data
train_inputs.extend(one_freq_input_ids)
train_labels.extend(one_freq_labels)
train_masks.extend(one_freq_attention_masks)
train_token_types.extend(one_freq_token_types)

# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
train_token_types = torch.tensor(train_token_types)

validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)
validation_token_types = torch.tensor(validation_token_types)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  


In [25]:
# Select a batch size for training. For fine-tuning with XLNet, the authors recommend a batch size of 32, 48, or 128. We will use 32 here to avoid memory issues.
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels, train_token_types)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels, validation_token_types)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [26]:
torch.save(validation_dataloader,'validation_data_loader')
torch.save(train_dataloader,'train_data_loader')

## Load Model & Set Params

Load the appropriate model below, each model already contains a single dense layer for classification on top.



```
BERT:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

XLNet:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=num_labels)

RoBERTa:
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=num_labels)
```



In [27]:
# Load model, the pretrained model will include a single linear classification layer on top for classification. 
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

Setting custom optimization parameters for the AdamW optimizer https://huggingface.co/transformers/main_classes/optimizer_schedules.html

In [28]:
# setting custom optimization parameters. You may implement a scheduler here as well.
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [29]:
optimizer = AdamW(optimizer_grouped_parameters,lr=2e-5,correct_bias=True)
# optimizer = AdamW(model.parameters(),lr=2e-5)  # Default optimization

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


## Train Model

In [30]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 10

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):

  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()

  # Tracking variables
  tr_loss = 0 #running loss
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, b_token_types = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()

    # # Forward pass for multiclass classification
    # outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    # loss = outputs[0]
    # logits = outputs[1]

    # Forward pass for multilabel classification
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]
    loss_func = BCEWithLogitsLoss() 
    loss = loss_func(logits.view(-1,num_labels),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation
    # loss_func = BCELoss() 
    # loss = loss_func(torch.sigmoid(logits.view(-1,num_labels)),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation
    train_loss_set.append(loss.item())    

    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    # scheduler.step()
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))

###############################################################################

  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Variables to gather full output
  logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

  # Predict
  for i, batch in enumerate(validation_dataloader):
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, b_token_types = batch
    with torch.no_grad():
      # Forward pass
      outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      b_logit_pred = outs[0]
      pred_label = torch.sigmoid(b_logit_pred)

      b_logit_pred = b_logit_pred.detach().cpu().numpy()
      pred_label = pred_label.to('cpu').numpy()
      b_labels = b_labels.to('cpu').numpy()

    tokenized_texts.append(b_input_ids)
    logit_preds.append(b_logit_pred)
    true_labels.append(b_labels)
    pred_labels.append(pred_label)

  # Flatten outputs
  pred_labels = [item for sublist in pred_labels for item in sublist]
  true_labels = [item for sublist in true_labels for item in sublist]

  # Calculate Accuracy
  threshold = 0.50
  pred_bools = [pl>threshold for pl in pred_labels]
  true_bools = [tl==1 for tl in true_labels]
  val_f1_accuracy = f1_score(true_bools,pred_bools,average='micro')*100
  val_flat_accuracy = accuracy_score(true_bools, pred_bools)*100

  print('F1 Validation Accuracy: ', val_f1_accuracy)
  print('Flat Validation Accuracy: ', val_flat_accuracy)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Train loss: 0.41844554571434855


Epoch:  10%|█         | 1/10 [01:03<09:29, 63.23s/it]

F1 Validation Accuracy:  0.0
Flat Validation Accuracy:  13.839285714285715
Train loss: 0.2208667874801904


Epoch:  20%|██        | 2/10 [02:05<08:22, 62.84s/it]

...

F1 Validation Accuracy:  91.2718204488778
Flat Validation Accuracy:  86.16071428571429
Train loss: 0.05408835818525404


Epoch:  90%|█████████ | 9/10 [09:23<01:02, 62.47s/it]

F1 Validation Accuracy:  91.85185185185185
Flat Validation Accuracy:  87.5
Train loss: 0.046515847905538976


Epoch: 100%|██████████| 10/10 [10:25<00:00, 62.56s/it]

F1 Validation Accuracy:  92.15686274509804
Flat Validation Accuracy:  87.94642857142857


In [31]:
torch.save(model.state_dict(), 'sriram_bert_for_sequence_tagging')

## Load and Preprocess Test Data

In [32]:
test_df = pd.read_excel('/content/test.xlsx')

for i in relations2index.keys():

  test_df[f'{i}'] = [0]*len(test_df)

test_df = test_df.drop(['nan'], axis=1)

# test_labels_df = pd.read_csv('/content/test.xlsx')
# test_df = test_df.merge(test_labels_df, on='id', how='left')
test_label_cols = list(test_df.columns[1:])
print('Null values: ', test_df.isnull().values.any()) #should not be any null sentences or labels
print('Same columns between train and test: ', label_cols == test_label_cols) #columns should be the same
test_df.head()

Null values:  False
Same columns between train and test:  True


,utterances,movie.production_companies,movie.gross_revenue,movie.initial_release_date,movie.country,gr.amount,person.date_of_birth,movie.locations,movie.subjects,actor.gender,movie.directed_by,movie.starring.character,movie.music,movie.genre,movie.language,movie.rating,movie.starring.actor,movie.produced_by,movie.estimated_budget
0,star of thor,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,who is in the movie the campaign,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,list the cast of the movie the campaign,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,who was in twilight,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,who is in vulguria,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [33]:
test_df = test_df[~test_df[test_label_cols].eq(-1).any(axis=1)] #remove irrelevant rows/comments with -1 values
test_df['one_hot_labels'] = list(test_df[test_label_cols].values)
test_df.head()

,utterances,movie.production_companies,movie.gross_revenue,movie.initial_release_date,movie.country,gr.amount,person.date_of_birth,movie.locations,movie.subjects,actor.gender,movie.directed_by,movie.starring.character,movie.music,movie.genre,movie.language,movie.rating,movie.starring.actor,movie.produced_by,movie.estimated_budget,one_hot_labels
0,star of thor,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,who is in the movie the campaign,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,list the cast of the movie the campaign,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,who was in twilight,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,who is in vulguria,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [35]:
# Gathering input data
test_labels = list(test_df.one_hot_labels.values)
test_comments = list(test_df.utterances.values)

In [36]:
# Encoding input data
test_encodings = tokenizer.batch_encode_plus(test_comments,max_length=max_length,pad_to_max_length=True)
test_input_ids = test_encodings['input_ids']
test_token_type_ids = test_encodings['token_type_ids']
test_attention_masks = test_encodings['attention_mask']

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [37]:
# Make tensors out of data
test_inputs = torch.tensor(test_input_ids)
test_labels = torch.tensor(test_labels)
test_masks = torch.tensor(test_attention_masks)
test_token_types = torch.tensor(test_token_type_ids)
# Create test dataloader
test_data = TensorDataset(test_inputs, test_masks, test_labels, test_token_types)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)
# Save test dataloader
torch.save(test_dataloader,'test_data_loader')

## Prediction and Metics

In [ ]:
# Test

# Put model in evaluation mode to evaluate loss on the validation set
model.eval()

#track variables
logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

# Predict
for i, batch in enumerate(test_dataloader):
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  # print('batch:', batch)
  b_input_ids, b_input_mask, b_labels, b_token_types = batch
  with torch.no_grad():
    # Forward pass
    outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    b_logit_pred = outs[0]
    pred_label = torch.sigmoid(b_logit_pred)

    b_logit_pred = b_logit_pred.detach().cpu().numpy()
    pred_label = pred_label.to('cpu').numpy()
    b_labels = b_labels.to('cpu').numpy()

  tokenized_texts.append(b_input_ids)
  logit_preds.append(b_logit_pred)
  true_labels.append(b_labels)
  pred_labels.append(pred_label)

# Flatten outputs
tokenized_texts = [item for sublist in tokenized_texts for item in sublist]
pred_labels = [item for sublist in pred_labels for item in sublist]

true_labels = [item for sublist in true_labels for item in sublist]

# Converting flattened binary values to boolean values
true_bools = [tl==1 for tl in true_labels]

We need to threshold our sigmoid function outputs which range from [0, 1]. Below I use 0.50 as a threshold.

In [39]:
pred_bools = [pl>0.50 for pl in pred_labels] #boolean output after thresholding

# Print and save classification report
print('Test F1 Accuracy: ', f1_score(true_bools, pred_bools,average='micro'))
print('Test Flat Accuracy: ', accuracy_score(true_bools, pred_bools),'\n')
clf_report = classification_report(true_bools,pred_bools,target_names=test_label_cols)
pickle.dump(clf_report, open('classification_report.txt','wb')) #save report
print(clf_report)

Test F1 Accuracy:  0.0
Test Flat Accuracy:  0.2069317023445464 

                            precision    recall  f1-score   support

movie.production_companies       0.00      0.00      0.00         0
       movie.gross_revenue       0.00      0.00      0.00         0
movie.initial_release_date       0.00      0.00      0.00         0
             movie.country       0.00      0.00      0.00         0
                 gr.amount       0.00      0.00      0.00         0
      person.date_of_birth       0.00      0.00      0.00         0
           movie.locations       0.00      0.00      0.00         0
            movie.subjects       0.00      0.00      0.00         0
              actor.gender       0.00      0.00      0.00         0
         movie.directed_by       0.00      0.00      0.00         0
  movie.starring.character       0.00      0.00      0.00         0
               movie.music       0.00      0.00      0.00         0
               movie.genre       0.00      0.00   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMe

## Output Dataframe

In [52]:
idx2label = dict(zip(range(len(label_cols)),label_cols))
print(idx2label)

{0: 'movie.production_companies', 1: 'movie.gross_revenue', 2: 'movie.initial_release_date', 3: 'movie.country', 4: 'gr.amount', 5: 'person.date_of_birth', 6: 'movie.locations', 7: 'movie.subjects', 8: 'actor.gender', 9: 'movie.directed_by', 10: 'movie.starring.character', 11: 'movie.music', 12: 'movie.genre', 13: 'movie.language', 14: 'movie.rating', 15: 'movie.starring.actor', 16: 'movie.produced_by', 17: 'movie.estimated_budget'}


In [53]:
# Getting indices of where boolean one hot vector true_bools is True so we can use idx2label to gather label names
true_label_idxs, pred_label_idxs=[],[]
for vals in true_bools:
  true_label_idxs.append(np.where(vals)[0].flatten().tolist())
for vals in pred_bools:
  pred_label_idxs.append(np.where(vals)[0].flatten().tolist())

In [55]:
# Gathering vectors of label names using idx2label
true_label_texts, pred_label_texts = [], []
for vals in true_label_idxs:
  if vals:
    true_label_texts.append([idx2label[val] for val in vals])
  else:
    true_label_texts.append(vals)

for vals in pred_label_idxs:
  if vals:
    pred_label_texts.append([idx2label[val] for val in vals])
  else:
    pred_label_texts.append(vals)

In [56]:
# Decoding input ids to comment text
comment_texts = [tokenizer.decode(text,skip_special_tokens=True,clean_up_tokenization_spaces=False) for text in tokenized_texts]

In [57]:
# Converting lists to df
comparisons_df = pd.DataFrame({'comment_text': comment_texts, 'true_labels': true_label_texts, 'pred_labels':pred_label_texts})
comparisons_df.to_csv('comparisons.csv')
comparisons_df.head()

,comment_text,true_labels,pred_labels
0,star of thor,[],[movie.starring.actor]
1,who is in the movie the campaign,[],[movie.starring.actor]
2,list the cast of the movie the campaign,[],[movie.starring.actor]
3,who was in twilight,[],[movie.starring.actor]
4,who is in vulguria,[],[]


In [59]:
comparisons_df.to_csv('abc.csv')

## Bonus - Optimizing threshold value for micro F1 score

Doing this may result in a trade offs between precision, flat accuracy and micro F1 accuracy. You may tune the threshold however you want.

In [ ]:
# Calculate Accuracy - maximize F1 accuracy by tuning threshold values. First with 'macro_thresholds' on the order of e^-1 then with 'micro_thresholds' on the order of e^-2

macro_thresholds = np.array(range(1,10))/10

f1_results, flat_acc_results = [], []
for th in macro_thresholds:
  pred_bools = [pl>th for pl in pred_labels]
  test_f1_accuracy = f1_score(true_bools,pred_bools,average='micro')
  test_flat_accuracy = accuracy_score(true_bools, pred_bools)
  f1_results.append(test_f1_accuracy)
  flat_acc_results.append(test_flat_accuracy)

best_macro_th = macro_thresholds[np.argmax(f1_results)] #best macro threshold value

micro_thresholds = (np.array(range(10))/100)+best_macro_th #calculating micro threshold values

f1_results, flat_acc_results = [], []
for th in micro_thresholds:
  pred_bools = [pl>th for pl in pred_labels]
  test_f1_accuracy = f1_score(true_bools,pred_bools,average='micro')
  test_flat_accuracy = accuracy_score(true_bools, pred_bools)
  f1_results.append(test_f1_accuracy)
  flat_acc_results.append(test_flat_accuracy)

best_f1_idx = np.argmax(f1_results) #best threshold value

# Printing and saving classification report
print('Best Threshold: ', micro_thresholds[best_f1_idx])
print('Test F1 Accuracy: ', f1_results[best_f1_idx])
print('Test Flat Accuracy: ', flat_acc_results[best_f1_idx], '\n')

best_pred_bools = [pl>micro_thresholds[best_f1_idx] for pl in pred_labels]
clf_report_optimized = classification_report(true_bools,best_pred_bools, target_names=label_cols)
pickle.dump(clf_report_optimized, open('classification_report_optimized.txt','wb'))
print(clf_report_optimized)

Best Threshold:  0.6
Test F1 Accuracy:  0.6838089672413233
Test Flat Accuracy:  0.8889149395104567 

               precision    recall  f1-score   support

        toxic       0.60      0.83      0.70      6090
 severe_toxic       0.47      0.34      0.39       367
      obscene       0.68      0.74      0.71      3691
       threat       0.52      0.63      0.57       211
       insult       0.73      0.63      0.68      3427
identity_hate       0.79      0.41      0.54       712

    micro avg       0.65      0.73      0.68     14498
    macro avg       0.63      0.60      0.60     14498
 weighted avg       0.66      0.73      0.68     14498
  samples avg       0.07      0.07      0.07     14498



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
